In [37]:
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation
import subprocess
import os
import pyhmmer
from pyhmmer import hmmscan as hmmscan
import pyrodigal
import pandas as pd

In [4]:
%%time

input_file = "/Users/michaeltisza/mike_tisza/github_repos/Cenote-Taker2/testcontigs_DNA_ct2.fasta"

orf_finder = pyrodigal.OrfFinder(meta = True)

for record in SeqIO.parse(input_file, "fasta"):
    for i, pred in enumerate(orf_finder.find_genes(bytes(record.seq))):
        print(f">{record.id}_{i+1}", file = open("/Users/michaeltisza/mike_tisza/sandbox/ct2_test2.faa", "a"))
        print(pred.translate(), file = open("/Users/michaeltisza/mike_tisza/sandbox/ct2_test2.faa", "a"))


#genes = orf_finder.find_genes(record)



CPU times: user 63.8 ms, sys: 5.76 ms, total: 69.5 ms
Wall time: 68.8 ms


In [5]:
%%time

with pyhmmer.easel.SequenceFile("/Users/michaeltisza/mike_tisza/sandbox/ct2_test2.faa", digital=True) as seq_file:
    AAsequences = seq_file.read_block()



tester_hmmout = list(hmmscan(AAsequences, 
                        pyhmmer.plan7.HMMFile("/Users/michaeltisza/mike_tisza/sandbox/virus_specific_baits_plus_missed6a"), 
                        cpus=4))

CPU times: user 1min 44s, sys: 5.13 s, total: 1min 49s
Wall time: 39.5 s


In [15]:
#%%time

tester_hmmout = list(hmmscan(AAsequences, 
                        pyhmmer.plan7.HMMFile("/Users/michaeltisza/mike_tisza/sandbox/virus_specific_baits_plus_missed6a"), 
                        cpus=10))

In [60]:
#%%timeit

hmmscan_list = []

for model in tester_hmmout:
    quer1 = model.query_name.decode()
    for hit in model:
        target_name = hit.name.decode()
        target_acc = hit.accession
        full_seq_evalue = "%.2g" % hit.evalue
        hmmscan_list.append([quer1, target_name, full_seq_evalue])


hmmscan_df = pd.DataFrame(hmmscan_list, columns=["query", "target", "evalue"])\
    .sort_values('evalue').drop_duplicates('query')

In [ ]:
AA2 = SeqRecord